In [1]:
import devito
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
Lx = 100.0   # width of the mantle in the x direction []
Ly = 100.0   # thickness of the mantle in the y direction []
nx = 401     # number of points in the x direction
ny = 401     # number of points in the y direction
dx = Lx / (nx - 1)  # grid spacing in the x direction []
dy = Ly / (ny - 1)  # grid spacing in the y direction []
g = 9.81  # gravity acceleration [m/s^2]
alpha = 0.025 # friction coefficient for natural channels in good condition
# Maximum wave propagation time [s]
Tmax = 0.2
dt = 1/4500.
nt = (int)(Tmax/dt)

In [3]:
from devito import Grid, TimeFunction, Function

grid = Grid(shape=(ny, nx), extent=(Ly, Lx))
eta = TimeFunction(name='eta', grid=grid, space_order=2)
M = TimeFunction(name='M', grid=grid, space_order=2)
N = TimeFunction(name='N', grid=grid, space_order=2)
h = Function(name='h', grid=grid)
M

M(t, x, y)

In [4]:
D = eta + h
frictionTerm = g * alpha**2/D**(7/3)*(M**2+N**2)**(-1/2)
print(frictionTerm)

0.00613125*(M(t, x, y)**2 + N(t, x, y)**2)**(-0.5)*(eta(t, x, y) + h(x, y))**(-2.33333333333333)


In [5]:
import numpy
# Define the locations along a gridline.
x = numpy.linspace(0.0, Lx, num=nx)
y = numpy.linspace(0.0, Ly, num=ny)

# Define initial eta, M, N
X, Y = numpy.meshgrid(x,y) # coordinates X,Y required to define eta, h, M, N

# Define constant ocean depth profile h = 50 m
h.data[:] = 50 * numpy.ones_like(X)

# Define initial eta Gaussian distribution [m]
eta0 = 0.5 * numpy.exp(-((X-50)**2/10)-((Y-50)**2/10))

# Define initial M and N
M0 = 100. * eta0
N0 = 0. * M0

eta.data[:] = eta0.copy()
M.data[:]   = M0.copy()
N.data[:]   = N0.copy()

In [6]:
from devito import Eq, solve
pde_eta = Eq(eta.dt + M.dx + N.dy , subdomain=grid.interior)
pde_M   = Eq(M.dt + (M**2/D).dx + (M*N/D).dy + g*D*eta.dx + frictionTerm*M, subdomain=grid.interior)
pde_N   = Eq(N.dt + (M*N/D).dx + (N**2/D).dy + g*D*eta.dy + frictionTerm*N, subdomain=grid.interior)

stencil_eta = solve(pde_eta, eta.forward)
stencil_M   = solve(pde_M, M.forward)
stencil_N   = solve(pde_N, N.forward)

update_eta  = Eq(eta.forward, stencil_eta)
update_M    = Eq(M.forward, stencil_M)
update_N    = Eq(N.forward, stencil_M)

update_eta

Eq(eta(t + dt, x, y), dt*(-Derivative(M(t, x, y), x) - Derivative(N(t, x, y), y) + eta(t, x, y)/dt))

In [7]:
from devito import Operator

optime = Operator([update_eta, update_M, update_N])

In [8]:
optime.arguments()

ValueError: No value found for parameter dt

In [ ]:
plt.imshow(eta.data[0])

In [ ]:
plt.imshow(M.data[0])

In [ ]:
plt.imshow(N.data[0])

In [ ]:
#from devito import configuration
# Silence non-essential outputs from the solver.
#configuration['log-level'] = 'ERROR'

# This is the time loop.
#for step in range(0,nt):
optime(time_m=0, time_M=1, dt=dt)
plt.imshow(eta.data[0])

In [ ]:
print(eta.data)

In [ ]:
M.data